# Basic Map with Folium

* [examples](http://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/)
* [data: countries geojson](https://github.com/johan/world.geo.json/tree/master/countries)

In [1]:
!python --version

Python 3.6.1 :: Continuum Analytics, Inc.


In [2]:
import folium

folium.__version__

'0.3.0'

## basic map tiles

In [3]:
import os
import pandas as pd
import numpy as np

ph_coord=[13, 122]

m = folium.Map(location=ph_coord, 
                     #tiles='Stamen Toner',
                     #tiles='cartodbpositron'
                     zoom_start=6)

if not os.path.exists('results'):
    os.makedirs('results')
    
m.save(os.path.join('results', 'out0.html'))
m

## Polygon

### Add geojson data

* high resolution map of boundary is shown [here](https://github.com/benjiao/philippine-boundaries/blob/master/regions/regions.geojson)

In [4]:
import json, urllib.request

url_low_res='https://raw.githubusercontent.com/johan/world.geo.json/master/countries/PHL.geo.json'
url_hi_res='https://raw.githubusercontent.com/benjiao/philippine-boundaries/master/regions/regions.geojson'

with urllib.request.urlopen(url_low_res) as url:
    geojson_data = json.loads(url.read().decode())
    #print(geojson_data)  

In [5]:
geojson_data.keys()

dict_keys(['type', 'features'])

In [6]:
geojson_data['features'][0].keys()

dict_keys(['type', 'id', 'properties', 'geometry'])

In [7]:
[geojson_data['features'][i] for i in range(len(geojson_data['features']))]

[{'geometry': {'coordinates': [[[[126.376814, 8.414706],
      [126.478513, 7.750354],
      [126.537424, 7.189381],
      [126.196773, 6.274294],
      [125.831421, 7.293715],
      [125.363852, 6.786485],
      [125.683161, 6.049657],
      [125.396512, 5.581003],
      [124.219788, 6.161355],
      [123.93872, 6.885136],
      [124.243662, 7.36061],
      [123.610212, 7.833527],
      [123.296071, 7.418876],
      [122.825506, 7.457375],
      [122.085499, 6.899424],
      [121.919928, 7.192119],
      [122.312359, 8.034962],
      [122.942398, 8.316237],
      [123.487688, 8.69301],
      [123.841154, 8.240324],
      [124.60147, 8.514158],
      [124.764612, 8.960409],
      [125.471391, 8.986997],
      [125.412118, 9.760335],
      [126.222714, 9.286074],
      [126.306637, 8.782487],
      [126.376814, 8.414706]]],
    [[[123.982438, 10.278779],
      [123.623183, 9.950091],
      [123.309921, 9.318269],
      [122.995883, 9.022189],
      [122.380055, 9.713361],
      [122.586

In [8]:
geojson_data['features'][0]['geometry'].keys()

dict_keys(['type', 'coordinates'])

In [9]:
import pandas as pd
df=pd.DataFrame(geojson_data['features'][0]['geometry'])
df.head()

,coordinates,type
0,"[[[126.376814, 8.414706], [126.478513, 7.75035...",MultiPolygon
1,"[[[123.982438, 10.278779], [123.623183, 9.9500...",MultiPolygon
2,"[[[118.504581, 9.316383], [117.174275, 8.3675]...",MultiPolygon
3,"[[[121.883548, 11.891755], [122.483821, 11.582...",MultiPolygon
4,"[[[125.502552, 12.162695], [125.783465, 11.046...",MultiPolygon


### Overlay polygon

In [10]:
import folium

ph=[13, 122]
m = folium.Map(location=ph_coord, 
                     #tiles='Stamen Toner',
                     zoom_start=6)

folium.GeoJson(
    geojson_data,
).add_to(m)

m.save(os.path.join('results', 'out1.html'))
m

In [11]:
def geojson_to_pandas(geojson):
    df=pd.DataFrame(geojson['features'][0]['geometry'])
    return df

def plot_boundary_map(data,coordinate=[13, 122],tiles=None,style=None):
    assert isinstance(data,dict), 'data should be in geojson format!'
    m = folium.Map(location=coordinate, 
                     tiles=tiles,
                     zoom_start=6)
    folium.GeoJson(data,style_function=lambda feature: style).add_to(m)
    return m

Ipynb data rate is exceeded. See [this](https://github.com/jupyter/notebook/issues/2287) for help.

## Colormap

In [12]:
#num of polygons
regions=len(geojson_data['features'][0]['geometry']['coordinates'])

In [13]:
df=geojson_to_pandas(geojson_data)
#add column with dummy data (e.g. population)
df['dummy'] = np.random.random_sample(regions)*1e6
df['label'] = df['dummy'].apply(lambda x: '#008000' if x==np.max(x) else '#ff0000')
df

,coordinates,type,dummy,label
0,"[[[126.376814, 8.414706], [126.478513, 7.75035...",MultiPolygon,692239.748572,#008000
1,"[[[123.982438, 10.278779], [123.623183, 9.9500...",MultiPolygon,643363.270646,#008000
2,"[[[118.504581, 9.316383], [117.174275, 8.3675]...",MultiPolygon,604364.336278,#008000
3,"[[[121.883548, 11.891755], [122.483821, 11.582...",MultiPolygon,906041.377660,#008000
4,"[[[125.502552, 12.162695], [125.783465, 11.046...",MultiPolygon,993234.204979,#008000
5,"[[[121.527394, 13.06959], [121.26219, 12.20556...",MultiPolygon,342860.751224,#008000
6,"[[[121.321308, 18.504065], [121.937601, 18.218...",MultiPolygon,238544.203830,#008000


In [14]:
style = {'fillColor': '#008000', #df['label'].values,
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
        }
m=plot_boundary_map(geojson_data,style=style)
m
#m.save(os.path.join('results', 'out2.html'))

## Heatmap

In [15]:
#generate dummy data
np.random.seed(123)

mnl_coord=[14.5,120.9]
dummy_data = (np.random.normal(size=(10, 3)) *
        np.array(np.ones(3)) +
        np.array(np.concatenate((mnl_coord,[1])))).tolist()

In [16]:
#coordinates, value
dummy_data[:10]

[[13.414369396699438, 121.89734544658359, 1.282978498051992],
 [12.993705286081909, 120.32139974803147, 2.651436537097151],
 [12.073320756606925, 120.47108737114382, 2.2659362587055343],
 [13.633259597734899, 120.22111384837795, 0.9052910310631088],
 [15.991389626124288, 120.26109800331535, 0.5560180403539345],
 [14.065648724381482, 123.10593008272546, 3.1867860889737867],
 [15.504053897878878, 121.28618639917487, 1.7373685758962423],
 [15.990732028150799, 119.96416613159761, 2.1758290447821036],
 [13.246119332250988, 120.2622484975466, 1.9071051958003014],
 [13.07131929977403, 120.75993127981134, 0.13824510414031455]]

In [17]:
from folium.plugins.heat_map import HeatMap

mnl_coord=[14.5,120.9]
m = folium.Map(ph_coord, 
               tiles='stamentoner', 
               #tiles='cartodbpositron',
               zoom_start=6)

hm = HeatMap(dummy_data)

hm.add_to(m)
m